In [21]:
import os


class Para:
    read_folder_path = 'data/mt_space_1d_rate_20d_22-22_raw'
    write_folder_path = 'data/mt_space_1d_rate_20d_22-22_pre'

    feature_column_start_name = 'ep_ratio_ttm'
    feature_column_end_name = 'BR'

    month_num = len(os.listdir(read_folder_path))
para = Para()
print(para.month_num)

31


In [22]:
import time
import numpy as np
import pandas as pd


time_begin = time.time()

df = pd.DataFrame()

for i_csv in range(para.month_num):

    df_curr_month = pd.read_csv(para.read_folder_path +'/'+ str(i_csv) + '.csv', index_col='order_book_id')

    df_curr_month['date'] = pd.to_datetime(df_curr_month['date'])

    df = pd.concat([df, df_curr_month])

df

,board_type,sector_code,yield_rate,month,date,ep_ratio_ttm,pb_ratio_ttm,sp_ratio_ttm,MACD_DIFF,MACD_DEA,...,RSI10,SY,BIAS20,VOL30,VOL60,VOL120,VOLT20,VOLT60,AR,BR
order_book_id,,,,,,,,,,,,,,,,,,,,,
600519.XSHG,MainBoard,ConsumerStaples,-0.103465,0,2022-01-04,0.019457,15.322225,0.039718,35.040754,49.821406,...,49.040205,66.666667,-1.607371,0.313692,0.281185,0.342577,54.002066,125.670084,101.686730,122.345919
600519.XSHG,MainBoard,ConsumerStaples,-0.063251,1,2022-01-05,0.019719,15.118823,0.040252,28.243263,45.505786,...,48.264738,55.555556,-3.071912,0.309078,0.280892,0.342411,47.160473,125.961143,97.876311,112.119415
600519.XSHG,MainBoard,ConsumerStaples,-0.055100,2,2022-01-06,0.020135,14.806736,0.041100,19.262857,40.257209,...,42.544317,44.444444,-4.934375,0.308357,0.281111,0.343259,51.939002,126.108718,95.856099,108.965634
600519.XSHG,MainBoard,ConsumerStaples,-0.047389,3,2022-01-07,0.020552,14.506301,0.041952,8.798974,33.965571,...,28.258745,33.333333,-6.539335,0.305721,0.281535,0.343251,60.995399,126.070048,95.781897,101.917388
600519.XSHG,MainBoard,ConsumerStaples,-0.049507,4,2022-01-10,0.020301,14.685576,0.041440,2.415034,27.655473,...,18.256464,44.444444,-5.101151,0.304917,0.281676,0.343176,65.810594,126.103253,103.146520,103.711137
600519.XSHG,MainBoard,ConsumerStaples,-0.025730,5,2022-01-11,0.020567,14.495470,0.041983,-4.644348,21.195519,...,20.341138,33.333333,-5.926274,0.299577,0.275391,0.341855,70.986457,124.816800,95.977828,93.322512
600519.XSHG,MainBoard,ConsumerStaples,-0.037850,6,2022-01-12,0.020290,14.693046,0.041418,-8.012304,15.353964,...,25.013137,44.444444,-4.219177,0.297069,0.273687,0.341328,70.948727,123.742123,103.392277,98.254576
600519.XSHG,MainBoard,ConsumerStaples,0.005113,7,2022-01-13,0.021259,14.023679,0.043395,-17.708083,8.741564,...,25.578410,33.333333,-8.005644,0.305161,0.277413,0.343099,78.395351,123.328291,94.611640,87.287103
600519.XSHG,MainBoard,ConsumerStaples,0.021425,8,2022-01-14,0.021377,13.946068,0.043637,-25.931520,1.806956,...,16.598150,33.333333,-7.946436,0.307522,0.278250,0.343487,85.122736,123.052742,86.733175,78.405775


In [23]:
# Replace infinite updated data with nan
df.replace(to_replace=[np.inf, -np.inf], value=np.nan, inplace=True)

# 剔除含有 na 的行
df_dropna = df.dropna(axis=0, how='any')

# 选择特征列和标签列
df_dropna_y = df_dropna[['board_type', 'sector_code', 'month', 'date']]
df_dropna_features = df_dropna.loc[:, para.feature_column_start_name:para.feature_column_end_name]
df_dropna_features = pd.concat([df_dropna['yield_rate'], df_dropna_features], axis=1)

# 中位数去极值
df_dropna_features = df_dropna_features.clip(
    lower=df_dropna_features.median(axis=0) - 5 * df_dropna_features.mad(axis=0),
    upper=df_dropna_features.median(axis=0) + 5 * df_dropna_features.mad(axis=0),
    axis=1)

# 标准化
df_dropna_features = (df_dropna_features - df_dropna_features.mean()) / df_dropna_features.std(ddof=0)

# 拼接标签列和特征列
df_dropna_standard = pd.concat([df_dropna_y, df_dropna_features], axis=1)

# 写入文件
df_dropna_standard.to_csv(para.write_folder_path + '/0.csv')

print()
print('Done!')


Done!
